# Get PubMed Gene Info

So Pubmed is a nice website which has many databases all working together to fulfill search queries.

The one I care about is given a human gene name --> get the PubMed gene-id --> use the gene_id to get the gene page --> use the gene page to get the sequence information from genbank.

The genbank does not have an api & all their data files can be found through [their ftp server](https://ftp.ncbi.nlm.nih.gov/genbank/release.notes/). However, there is an issue as human and plant and all sorts of organisms are mixed together. Parsing this will be a pain :/

I will opt to be an asshole & use the human apis as at least I can figure out how they work & they get me to the solution.

# Imports and Globals 🌎

In [153]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import regex as re
from pickle import dump,load
import sys

# Make test Request to Gene Cards

In [2]:
selected_gene = "TM6SF2"
genecard_url = 'https://www.genecards.org/cgi-bin/carddisp.pl?gene={}'

selected_url = genecard_url.format(selected_gene)

payload={}

UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]
headers = {
  'user-agent':ua
}

response = requests.request("GET", selected_url, headers=headers, data=payload)

#print(response.text)

soup = BeautifulSoup(response.text, 'html.parser')
all_divs = soup.findAll("section",id="summaries")
print(len(all_divs))
#sub_soup = BeautifulSoup(all_divs[0],"html.parser")
all_subsections = all_divs[0].findAll("div",class_="gc-subsection")
print(len(all_subsections))
print(all_subsections[0])

h1 = soup.findAll("h1",id="geneSymbol")
if h1 and h1[0]:
  print(h1[0].findAll("small"))

1
5
<div class="gc-subsection">
<div class="gc-subsection-header">
<h3>Entrez Gene Summary for TM6SF2 Gene</h3>
<a class="gc-ga-link glyphicon glyphicon-new-window" data-ga-action="ENT" data-ga-source-accession="53345" href="https://www.ncbi.nlm.nih.gov/gene/53345#summary" target="_blank" title="See NCBI Entrez Gene entry for TM6SF2"></a>
</div>
<ul class="list-unstyled">
<li>
<p>Enables identical protein binding activity. Involved in regulation of lipid metabolic process. Located in endoplasmic reticulum membrane and endoplasmic reticulum-Golgi intermediate compartment membrane. [provided by Alliance of Genome Resources, Apr 2022]</p>
</li>
</ul>
</div>
[<small>
                    Gene
                        <span id="geneDescription" title="Transmembrane 6 Superfamily Member 2">- Transmembrane 6 Superfamily Member 2</span>
</small>]


ok the requests & bs4 is setup correctly

# Make Test Request to PubMed

## Incremental Dev

In [140]:
response = requests.request("GET", "https://www.ncbi.nlm.nih.gov/gene/?term=MBTPS1", headers=headers, data=payload)

In [141]:
soup = BeautifulSoup(response.text, 'html.parser')
titles = soup.findAll("a",id="feat_gene_title")

In [142]:
titles

[<a data-ga-action="click_feat_title" data-ga-label="feat_gene_title" href="https://www.ncbi.nlm.nih.gov/gene/8720" id="feat_gene_title" ref="discoId=KnownItemSensor&amp;itemType=gene&amp;itemId=gene:8720&amp;linkPos=0">MBTPS1 – membrane bound transcription factor peptidase, site 1</a>]

In [35]:
taxons = soup.findAll("p",class_="ncbi-doc-authors")
taxons

[<p class="ncbi-doc-authors">
 <a data-ga-action="click_feat_taxname" data-ga-label="feat_taxname" href="https://www.ncbi.nlm.nih.gov/labs/data-hub/taxonomy/9606"><span data-item-id="taxonomy:9606" data-item-type="taxon"><i>Homo sapiens</i></span> (human)</a>
 </p>]

In [143]:
titles[0]["href"]

'https://www.ncbi.nlm.nih.gov/gene/8720'

In [144]:
titles[0].contents[0].split()[0]

'MBTPS1'

In [38]:
taxons[0].find("i").contents[0]

'Homo sapiens'

In [39]:
response2 = requests.request("GET", titles[0]["href"], headers=headers, data=payload)
#response.text

In [40]:
soup2 = BeautifulSoup(response2.text, 'html.parser')

In [41]:
soup2.findAll("a",title="Nucleotide GenBank report")[0]["href"]

'/nuccore/NC_000004.12?report=genbank&from=99564130&to=99623997'

In [42]:
response3 = requests.request("GET", "https://www.ncbi.nlm.nih.gov/"+soup2.findAll("a",title="Nucleotide GenBank report")[0]["href"], headers=headers, data=payload)
soup3 = BeautifulSoup(response3.text,"html.parser")

In [43]:
#response3.text
target_url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815579&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=19264366&to=19273301&strand=on&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000"

In [44]:
"568815579" in response3.text

False

In [45]:
i = response3.text.index("568815579")

ValueError: substring not found

In [46]:
response3.text[i-100:i+100]

'<meta name="ncbi_sessionid" content="CE89C68F32CD9541_0116SID" /><meta name="ncbi_uidlist" content="568815594" /><meta name="ncbi_filter" content="all" /><meta name="ncbi_stat" content="false" /><meta'

In [47]:
soup3.findAll("meta",{"name":"ncbi_uidlist"})[0]["content"]

'568815594'

In [84]:
soup3.findAll("meta",{"name":"ncbi_phid"})[0]["content"]

'CE89C68F32CD6C710000000000740027.m_23'

In [85]:
temp2 = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&=&from={}&to={}&retmode=html&ncbi_phid={}&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000"
temp2.format(ncbi_id,start,stop,"CE89C68F32CD6C710000000000740027")

'https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815594&db=nuccore&report=genbank&=&from=99564130&to=99623997&retmode=html&ncbi_phid=CE89C68F32CD6C710000000000740027&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000'

In [49]:
m = re.search(re.compile("from=(\d+)&to=(\d+)"),soup2.findAll("a",title="Nucleotide GenBank report")[0]["href"])
m.groups()

('99564130', '99623997')

In [57]:
target_url_template = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from={}&to={}&strand={}&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000"
ncbi_id = soup3.findAll("meta",{"name":"ncbi_uidlist"})[0]["content"]
start,stop = re.search(re.compile("from=(\d+)&to=(\d+)"),soup2.findAll("a",title="Nucleotide GenBank report")[0]["href"]).groups()
strand = "on" if "strand=true" in soup2.findAll("a",title="Nucleotide GenBank report")[0]["href"] else "off"
target_url_template.format(ncbi_id,start,stop,strand)

'https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815594&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=99564130&to=99623997&strand=off&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000'

In [59]:
response4 = requests.request("GET", target_url_template.format(ncbi_id,start,stop,strand), headers=headers, data=payload)

In [60]:
m = re.search(re.compile(r"CDS\s+join\(((?:\.|,|\d|\s)+)\)"),response4.text)
"".join(m.groups()[0].split())

'10781..10841,17776..17963,19245..19388,25514..25621,27106..27222,27522..27661,30604..30754,32938..33095,36436..36604,37478..37585,42619..42831,44637..44848,47014..47111,47203..47324,48784..49011,54845..54969,56932..57102,58548..58719'

In [61]:
soup4 = BeautifulSoup(response4.text,"html.parser")

In [102]:
seq = "".join(["".join(s.contents[0].split()) for s in soup4.findAll("span",class_="ff_line")])

In [103]:
seq[87-1:87-1+3]

'gtg'

In [104]:
len(seq)

59868

In [105]:
int(stop)-int(start)

59867

yay! I can get the nucleotide sequence of the gene & the coding regions of it.

## Turn into function

In [127]:
def get_bp_and_cds(gene):
    result = {}
    payload={}

    UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
           "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
           "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
           "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
           "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
           )

    ua = UAS[random.randrange(len(UAS))]
    headers = {
      'user-agent':ua
    }

    response1 = requests.request("GET", "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(gene), headers=headers, data=payload)
    soup1 = BeautifulSoup(response1.text, 'html.parser')
    titles = soup1.findAll("a",id="feat_gene_title")
    result["gene"] = titles[0].contents[0].split()[0]
    taxons = soup.findAll("p",class_="ncbi-doc-authors")
    result["organism"] = taxons[0].find("i").contents[0]
    result["gene_link"] = titles[0]["href"]
    
    response2 = requests.request("GET", titles[0]["href"], headers=headers, data=payload)
    soup2 = BeautifulSoup(response2.text, 'html.parser')
    result["gene_bank_url"] = "https://www.ncbi.nlm.nih.gov/"+soup2.findAll("a",title="Nucleotide GenBank report")[0]["href"]
    
    response3 = requests.request("GET", result["gene_bank_url"], headers=headers, data=payload)
    soup3 = BeautifulSoup(response3.text,"html.parser")
    soup3.findAll("meta",{"name":"ncbi_uidlist"})[0]["content"]
    result["ncbi_id"] = soup3.findAll("meta",{"name":"ncbi_uidlist"})[0]["content"]
    result["start"],result["stop"] = re.search(re.compile("from=(\d+)&to=(\d+)"),result["gene_bank_url"]).groups()
    result["strand"] = "on" if "strand=true" in result["gene_bank_url"] else "off"
    result["ncbi_phid"] = soup3.findAll("meta",{"name":"ncbi_phid"})[0]["content"].split()[0]
    
    target_url_template = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from={}&to={}&strand={}&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000"
    #target_url_template2 = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&=&from={}&to={}&retmode=html&ncbi_phid={}&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000"
    result["genbank_jquery"] = target_url_template.format(result["ncbi_id"],result["start"],result["stop"],result["strand"])
    print(result["gene_bank_url"])
    print(result["genbank_jquery"])
    response4 = requests.request("GET", result["genbank_jquery"], headers=headers, data=payload)
    print(len(response4.text))
    m = re.search(re.compile(r"CDS\s+join\(((?:&|l|t|;|\.|,|\d|\s)+)\)"),response4.text)
    result["CDS"] = "".join(m.groups()[0].split())
    soup4 = BeautifulSoup(response4.text,"html.parser")
    result["seq"] = "".join(["".join(s.contents[0].split()) for s in soup4.findAll("span",class_="ff_line")])
    print("done!")
    return result

## Test function behavior

In [128]:
tm6 = get_bp_and_cds("TM6SF2")

https://www.ncbi.nlm.nih.gov//nuccore/NC_000019.10?report=genbank&from=19264366&to=19273301&strand=true
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815579&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=19264366&to=19273301&strand=on&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
33118
done!


In [133]:
tm6["seq"][87-1:100]

'atggacatcccgcc'

In [129]:
mttp = get_bp_and_cds("MTTP")

https://www.ncbi.nlm.nih.gov//nuccore/NC_000004.12?report=genbank&from=99564130&to=99623997
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815594&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=99564130&to=99623997&strand=off&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
153894
done!


In [135]:
int(mttp["stop"])-int(mttp["start"])+1 == len(mttp["seq"])

True

In [136]:
mttp["CDS"]

'10781..10841,17776..17963,19245..19388,25514..25621,27106..27222,27522..27661,30604..30754,32938..33095,36436..36604,37478..37585,42619..42831,44637..44848,47014..47111,47203..47324,48784..49011,54845..54969,56932..57102,58548..58719'

In [138]:
mttp["seq"][10781-1:10781-1+3]

'atg'

In [122]:
ncan = get_bp_and_cds("NCAN")

https://www.ncbi.nlm.nih.gov//nuccore/NC_000019.10?report=genbank&from=19211958&to=19252233
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815579&db=nuccore&report=genbank&=&from=19211958&to=19252233&retmode=html&ncbi_phid=CE8B936032CD8FD100000000044F0185.m_23&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
112708
done!


In [130]:
trac = get_bp_and_cds("TRAC")

https://www.ncbi.nlm.nih.gov//nuccore/NC_000014.9?report=genbank&from=22547506&to=22552132
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815584&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=22547506&to=22552132&strand=off&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
22802
done!


In [131]:
trac

{'gene': 'TRAC',
 'organism': 'Homo sapiens',
 'gene_link': 'https://www.ncbi.nlm.nih.gov/gene/28755',
 'gene_bank_url': 'https://www.ncbi.nlm.nih.gov//nuccore/NC_000014.9?report=genbank&from=22547506&to=22552132',
 'ncbi_id': '568815584',
 'start': '22547506',
 'stop': '22552132',
 'strand': 'off',
 'ncbi_phid': 'CE8A6C5F32CDB6110000000003BE0150.m_23',
 'genbank_jquery': 'https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815584&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=22547506&to=22552132&strand=off&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000',
 'CDS': '&lt;1..274,2133..2177,3052..3159',
 'seq': 'atatccagaaccctgaccctgccgtgtaccagctgagagactctaaatccagtgacaagtctgtctgcctattcaccgattttgattctcaaacaaatgtgtcacaaagtaaggattctgatgtgtatatcacagacaaaactgtgctagacatgaggtctatggacttcaagagcaacagtgctgtggcctggagcaacaaatctgactttgcatgtgcaaacgccttcaacaacagcattattccagaagacaccttcttccccagcccaggtaagggcagctttggtgccttcgcaggctgtttccttgcttcaggaatggcca

In [111]:
22552132-22547506

4626

In [117]:
trac["seq"][:20]

'atatccagaaccctgaccct'

In [145]:
MBTPS1 = get_bp_and_cds("MBTPS1")

https://www.ncbi.nlm.nih.gov//nuccore/NC_000016.10?report=genbank&from=84053763&to=84116942&strand=true
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815582&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=84053763&to=84116942&strand=on&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
162759
done!


It works for 3 "common" genes, does it work for more?

One thing to note is that TRAC is kinda wack tho ... It does not start with the usual start codon "atg"

# Get PubMed for all Prabu Genes

In [146]:
gene_to_data = {}

In [147]:
with open("../data/genes.list","r") as f:
    for l in f:
        l = l.strip()
        l = l.split()
        l = l[0]
        if l not in gene_to_data:
            print("on {}".format(l))
            gene_to_data[l] = get_bp_and_cds(l)
print(len(gene_to_data))

on ADIPOQ
https://www.ncbi.nlm.nih.gov//nuccore/NC_000003.12?report=genbank&from=186842710&to=186858463
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815595&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=186842710&to=186858463&strand=off&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
52335
done!
on ZMPSTE24
https://www.ncbi.nlm.nih.gov//nuccore/NC_000001.11?report=genbank&from=40258236&to=40294180
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815597&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=40258236&to=40294180&strand=off&retmode=html&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000
99727
done!
on TRAC
https://www.ncbi.nlm.nih.gov//nuccore/NC_000014.9?report=genbank&from=22547506&to=22552132
https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=568815584&db=nuccore&report=genbank&conwithfeat=on&basic_feat=on&hide-cdd=on&from=22547506&to=22552132&strand=off&retmode=html&with

In [154]:
# sys.setrecursionlimit(100000)

In [155]:
# with open("../data/gene_to_data.pickle","wb+") as f:
#     dump(gene_to_data,f)

In [28]:
# https://www.ncbi.nlm.nih.gov/gene/?term=TM6SF2